In [1]:
!pip install plotly==5.6.0
!pip install pandas 
!pip install -U scikit-learn

In [15]:
import pandas as pd
import glob
import csv
import os
from functools import reduce
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
from umap import UMAP

In [31]:
#used to generate a matrix of all data found in hyprseq reads 
os.chdir("/Users/sakshamgupta/Desktop/HyprSeq_analysis_saksham/data/UMI_count")
root = "/Users/sakshamgupta/Desktop/HyprSeq_analysis_saksham/data/UMI_count"
allCounts = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if name.endswith("counts.txt"):
            allCounts.append(os.path.join(path, name))
            
geneFile = open("/Users/sakshamgupta/Desktop/HyprSeq_analysis_saksham/probe_index.txt", "r")
lines = geneFile.readlines()
geneNamesFinal = []
for i in lines:
    if i.startswith(">"):
        geneNamesFinal.append(i[1:])

finalColNames = []
for i in allCounts:
    unfiltNames = i.split("/EHyprSeq-",1)[1]
    Names = unfiltNames.split(".",1)[0]
    finalColNames.append(Names)
    
mergeArrayContents = [] 
for files in allCounts:
    data = open(files, "r")
    genes = data.readlines()
    finalCount = {"Genes":[],"Count":[]}
    for i in genes:
        finalCount["Genes"].append(i.split("\t")[0])
        finalCount["Count"].append(int(i.split("\t")[1]))
    mergeArrayContents.append(finalCount)
    
data_frames = []
for i in range(len(allCounts)):
    data_frames.append(pd.DataFrame(mergeArrayContents[i]))
    

    
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Genes'],
                                            how='outer'), data_frames).fillna(0)

df_merged.index = geneNamesFinal
del df_merged["Genes"]
df_merged.columns = finalColNames

print(df_merged)



/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T

                        NegT4-1_counts  Embryo-1-1_counts  Nih3t3-2-3_counts  \
Actb-exon-1_5prime\n               0.0               2712             1870.0   
Actb-exon-2_5prime\n               0.0               3532             2412.0   
Apoe-exon-1_5prime\n               0.0                  7                3.0   
Apoe-exon-5_5prime\n               0.0                404                4.0   
Apoe-exon-9_5prime\n               0.0                191                0.0   
...                                ...                ...                ...   
Tuba1a-exon-2_5prime\n             0.0                358              299.0   
Tuba1a-exon-3_5prime\n             0.0                507              544.0   
Wfdc2-exon-1_5prime\n              0.0                 34                4.0   
Wfdc2-exon-2_5prime\n              0.0                 30                3.0   
Wfdc2-exon-3_5prime\n              0.0                 65               13.0   

                        NegRo-2_counts 

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.

/var/folders/_1/v77zt5js6k783_zdyv8b691h0000gn/T/ipykernel_23045/1730785161.py:40: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Count_x'} in the result is deprecated and will raise a MergeError in a future version.



In [47]:
#used to make UMAP 
finalVar = geneNamesFinal[-1]
features = df_merged_transposed.loc[:, :finalVar]
df_merged_transposed['SampleNames'] = finalColNames

umap_2d = UMAP(n_components=2, init='random', random_state=0)
umap_3d = UMAP(n_components=3, init='random', random_state=0)

proj_2d = umap_2d.fit_transform(features)
proj_3d = umap_3d.fit_transform(features)

fig_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=df_merged_transposed.Samples, labels={'color': 'Samples'},
    #text =df_merged_transposed['SampleNames']
)
fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=df_merged_transposed.Samples, labels={'color': 'Samples'},
    #text =df_merged_transposed['SampleNames']
)


fig_2d.show()
fig_3d.show()


In [17]:
#generates PCA plots of data found in the reads of the HyPRseq data
df_merged_transposed = df_merged.T
transposedNames = []
for i in finalColNames:
    trueName = i.split("-")
    transposedNames.append(trueName[0])

    
df_merged_transposed['Samples'] = transposedNames



X = df_merged_transposed[geneNamesFinal]
pca = PCA(n_components=2)
components = pca.fit_transform(X)
fig = px.scatter(components, x=0, y=1, color = df_merged_transposed['Samples']) 
fig.update_layout(xaxis_title="PCA 1",yaxis_title="PCA 2",title="Standard PCA analysis of HyPRSeq Data")
fig.show()
logFig = px.scatter(components, x=0, y=1, color = df_merged_transposed['Samples'], log_y = True) 
logFig.update_layout(xaxis_title="PCA 1",yaxis_title="PCA 2",title="Logarithmic PCA analysis of HyPRSeq Data")
logFig.show()

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

threeDfig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df_merged_transposed['Samples'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PCA 1', '1': 'PCA 2', '2': 'PCA 3'}
)
threeDfig.show()

In [19]:
#used to generate comparative graph of RNAseq data from  with hyprseq data 
#Load in respective RNASeq data 
genesUtilized = []
for genes in geneNamesFinal:
    a = genes.split("-")
    if a[0] not in genesUtilized:
        genesUtilized.append(a[0])
RNAseqData = []
with open('/Users/sakshamgupta/Desktop/HyprSeq_analysis_saksham/data/GSE101748_GeneLevel_Normalized(CPM.and.TMM)_data.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        if row['gene_symbol'] in genesUtilized:
            a = float(row['GSM2714377'])
            b = float(row['GSM2714378'])
            c = float(row['GSM2714378'])
            
            geneLevel = max(a,b,c)
            geneLevel = int(geneLevel)            
            
            RNAseqData.append({"Genes":row['gene_symbol'], "RNAseq_expression":geneLevel})
RNAseqData = pd.DataFrame(RNAseqData)
RNAseqData = RNAseqData.sort_values(by=['Genes'])

df_truncated = [df_merged['Embryo-1-1_counts'],df_merged['Embryo-1-2_counts'], df_merged['Embryo-1-3_counts'],
                df_merged['Embryo-2-1_counts'],df_merged['Embryo-2-2_counts'],df_merged['Embryo-2-3_counts']]
df_truncated = pd.DataFrame(df_truncated)
df_truncated = df_truncated.T.groupby([s.split('-')[0] for s in df_truncated.T.index.values]).max()
df_truncated = df_truncated.rename_axis('Genes')

data_frames = []
data_frames.append(df_truncated)
data_frames.append(RNAseqData)
df_truncated['genes'] = genesUtilized
finalDF= reduce(lambda  left,right: pd.merge(left,right,on=['Genes'],
                                            how='outer'), data_frames).fillna(0)
for i in df_truncated:
    
    if i != "genes":
        finalDF[i] = finalDF[i] /finalDF[i].abs().max()
        finalDF["RNAseq_expression"] = finalDF["RNAseq_expression"] /finalDF["RNAseq_expression"].abs().max()
        fig = px.scatter(x = finalDF[i], y = finalDF["RNAseq_expression"],text =df_truncated['genes'], trendline="ols")
    else:
        fig = px.scatter(x = finalDF[i], y = finalDF["RNAseq_expression"])

    fig.update_layout(xaxis_title=i,yaxis_title="RNAseq_expression")
    fig.show()


        